In [1]:
!pip install agentpy owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.46-cp310-cp310-linux_x86_64.whl size=24073559 sha256=b891d2ad45f2d7bb67fccebaa7929da48be7ef62da192f7af8242987e6db01a6
  Stored in directory: /root/.cache/pip/wheels/bf/6f/ef/35df148baa59fc6b243aca4f101e17aed14b67dc1166b73fe0
Successfully built owlready2


In [2]:
import agentpy as ap
from owlready2 import *
from matplotlib import pyplot as plt
import IPython
import random

In [3]:
onto = get_ontology("file://onto.owl")

In [4]:
with onto:
    class Entity(Thing):
      pass

    class Drone(Thing):
      pass

    class Box(Thing):
      pass

    class Wall(Thing):
      pass

    class Stack(Thing):
      pass

    class World(Thing):
      pass

    class Place(Thing):
      pass

    class has_position(DataProperty, FunctionalProperty):
      domain = [Entity]
      range = [str]

    class has_place(ObjectProperty, FunctionalProperty):
      domain = [Entity]
      range = [Place]

In [5]:
class DroneAgent(ap.Agent):
    def see(self, grid):
        self.visited = set()
        self.drones = []
        self.boxes = []
        self.walls = []
        self.stacking = []
        vecinos = grid.neighbors(self,1)
        for vecino in vecinos:
          if vecino.agentType == 0:
            self.drones.append(Drone(has_place=Place(has_position=str(grid.positions[vecino]))))
          elif vecino.agentType == 1:
            self.boxes.append(Box(has_place=Place(has_position=str(grid.positions[vecino]))))
          elif vecino.agentType == 2:
            self.walls.append(Wall(has_place=Place(has_position=str(grid.positions[vecino]))))
          elif vecino.agentType == 3:
            self.stacking.append(Stack(has_place=Place(has_position=str(grid.positions[vecino]))))

    def next(self):
        for act in self.actions:
            for rule in self.rules:
                if rule(act):
                    act()
        pass

    def setup(self):
        self.agentType = 0
        self.direction = (-1, 0)
        self.lastDirection = None
        self.withBox = False
        self.rules = (self.rule_1, self.rule_2, self.rule_3, self.rule_4)
        self.actions = (self.forward, self.take, self.drop, self.turnRandom)
        pass

    def step(self):
        self.see(self.model.grid)
        self.next()
        pass

    def update(self):
        pass

    def end(self):
        pass

    #REGLAS
    def rule_1(self,act):
        validador = [False, False]
        if not self.withBox:
          if len(self.boxes) > 0:
            for box in self.boxes:
                if eval(box.has_place.has_position)[0] == self.model.grid.positions[self][0]-1 \
                and eval(box.has_place.has_position)[1] == self.model.grid.positions[self][1]:
                  self.look_up()
                  validador[0] = True
                elif eval(box.has_place.has_position)[0] == self.model.grid.positions[self][0]+1 \
                and eval(box.has_place.has_position)[1] == self.model.grid.positions[self][1]:
                  self.look_down()
                  validador[0] = True
                elif eval(box.has_place.has_position)[1] == self.model.grid.positions[self][1]+1 \
                and eval(box.has_place.has_position)[0] == self.model.grid.positions[self][0]:
                  self.look_right()
                  validador[0] = True
                elif eval(box.has_place.has_position)[1] == self.model.grid.positions[self][1]-1 \
                and eval(box.has_place.has_position)[0] == self.model.grid.positions[self][0]:
                  self.look_left()
                  validador[0] = True
        if act == self.take:
            validador[1] = True
        return sum(validador) == 2


    def rule_2(self,act):
        validador = [False, False]
        if self.withBox:
          if len(self.stacking) > 0:
            for stack in self.stacking:
                if eval(stack.has_place.has_position)[0] == self.model.grid.positions[self][0]-1 \
                and eval(stack.has_place.has_position)[1] == self.model.grid.positions[self][1]:
                  self.look_up()
                  validador[0] = True
                elif eval(stack.has_place.has_position)[0] == self.model.grid.positions[self][0]+1 \
                and eval(stack.has_place.has_position)[1] == self.model.grid.positions[self][1]:
                  self.look_down()
                  validador[0] = True
                elif eval(stack.has_place.has_position)[1] == self.model.grid.positions[self][1]+1 \
                and eval(stack.has_place.has_position)[0] == self.model.grid.positions[self][0]:
                  self.look_right()
                  validador[0] = True
                elif eval(stack.has_place.has_position)[1] == self.model.grid.positions[self][1]-1 \
                and eval(stack.has_place.has_position)[0] == self.model.grid.positions[self][0]:
                  self.look_left()
                  validador[0] = True
        if act == self.drop:
            validador[1] = True
        return sum(validador) == 2


    def rule_3(self, act):
      validador = [False, False]
      turned = False
      nextPos = self.model.grid.positions[self]
      nextPos = (nextPos[0] + self.direction[0], nextPos[1] + self.direction[1])

      # Verifica si hay paredes en la dirección actual
      if len(self.boxes) <= 0:
          if len(self.walls) > 0:
              for wall in self.walls:
                  if eval(wall.has_place.has_position) == nextPos:
                      self.turnRandom()
                      turned = True

          # Verifica si hay un stack en la dirección actual
          if len(self.stacking) > 0:
              for stack in self.stacking:
                  if eval(stack.has_place.has_position) == nextPos:
                      self.turnRandom()
                      turned = True

          if not turned:
              validador[0] = True
      else:
          validador[0] = True

      if act == self.forward:
          validador[1] = True

      return sum(validador) == 2


    def rule_4(self,act):
        validador = [False, False]
        if len(self.drones) > 0:
          for drone in self.drones:
            if eval(drone.has_place.has_position) == self.model.grid.positions[self] \
            or eval(drone.has_place.has_position)[0] == self.model.grid.positions[self][0]+1 \
            or eval(drone.has_place.has_position)[1] == self.model.grid.positions[self][1]+1:
              validador[0] = True
        if act == self.turnRandom:
            validador[1] = True
        return sum(validador) == 2

    #ACCIONES
    def look_up(self):
          self.lastDirection = self.direction
          self.direction = (-1, 0)

    def look_down(self):
          self.lastDirection = self.direction
          self.direction = (1, 0)

    def look_left(self):
          self.lastDirection = self.direction
          self.direction = (0, -1)


    def look_right(self):
          self.lastDirection = self.direction
          self.direction = (0, 1)

    def turnRandom(self):
        self.direction = random.choice([(-1, 0), (1, 0), (0, -1), (0, 1)])

    def take(self):
        isStack = False
        nextPos = self.model.grid.positions[self]
        nextPos = (nextPos[0] + self.direction[0], nextPos[1] + self.direction[1])
        if len(self.stacking) > 0:
          for stack in self.stacking:
            if eval(stack.has_place.has_position) == nextPos:
              isStack = True
              break
        if not isStack:
          for box in self.model.grid.agents:
            if box.agentType == 1:
              if box.pos == nextPos:
                self.model.grid.remove_agents(box)
                self.withBox = True
                break
          self.direction = self.lastDirection

    def drop(self):
        nextPos = self.model.grid.positions[self]
        nextPos = (nextPos[0] + self.direction[0], nextPos[1] + self.direction[1])
        stackPos = nextPos
        self.model.grid.add_agents(self.boxes, positions=nextPos, empty=True)
        self.withBox = False
        self.direction = self.lastDirection

    def forward(self):
          self.model.grid.move_by(self, self.direction)


In [6]:
class BoxAgent(ap.Agent):
    def setup(self):
        self.agentType = 1
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        if self.first_step:
          self.pos = self.model.grid.positions[self]
          self.first_step = False
        pass

In [7]:
class WallAgent(ap.Agent):
    def setup(self):
        self.agentType = 2
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        if self.first_step:
          self.pos = self.model.grid.positions[self]
          self.first_step = False
        pass

In [8]:
class StackingAgent(ap.Agent):
    def setup(self):
        self.agentType = 3
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        pass

In [9]:
class WarehouseModel(ap.Model):
    def setup(self):
        self.drones = ap.AgentList(self, len(self.p.drones), DroneAgent)
        self.boxes = ap.AgentList(self, len(self.p.boxes), BoxAgent)
        self.walls = ap.AgentList(self, len(self.p.walls), WallAgent)
        self.stacking = ap.AgentList(self, len(self.p.stacking), StackingAgent)
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.grid.add_agents(self.drones, positions=self.p.drones, empty=True)
        self.grid.add_agents(self.boxes, positions=self.p.boxes, empty=True)
        self.grid.add_agents(self.walls, positions=self.p.walls, empty=True)
        self.grid.add_agents(self.stacking, positions=self.p.stacking, empty=True)

    def step(self):
        self.drones.step()
        self.boxes.step()
        pass

    def update(self):
        pass

    def end(self):
        pass

In [10]:
def animation_plot(model, ax):
    agent_type_grid = model.grid.attr_grid('agentType')
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Vaccum Model \n Time-step: {model.t}, "
                f"dirty: {0}")

import json

    # Example JSON string received in the POST request
request_data = '''{
   "warehouse": [
    ["W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W"],
    ["W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", "", "W"],
    ["W", " ", " ", "B", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W"],
    ["W", " ", " ", " ", " ", " ", " ", " ", "W", " ", "D", " ", " ", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", "S", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", "D", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", "S", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", "D", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", "D", " ", "W"],
    ["W", " ", " ", " ", "W", "S", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W"]
  ]
}'''

# Parse the JSON data
data = json.loads(request_data)
warehouse = data['warehouse']

parameters = {
    'M': len(warehouse),
    'N': len(warehouse[0]),
    'steps': 100,  # Assuming you have some logic to determine steps
    'drones': [],
    'boxes': [],
    'walls': [],
    'stacking': []
}

# Process the matrix
for row_idx, row in enumerate(warehouse):
    for col_idx, cell in enumerate(row):
        if cell == 'W':
            parameters['walls'].append((row_idx, col_idx))
        elif cell == 'B':
            parameters['boxes'].append((row_idx, col_idx))
        elif cell == 'D':
            parameters['drones'].append((row_idx, col_idx))
        elif cell == 'S':
            parameters['stacking'].append((row_idx, col_idx))

In [11]:
fig, ax = plt.subplots()
model = WarehouseModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())